In [1]:
!pip install pandas numpy transformers sentence-transformers faiss-cpu pinecone-client elasticsearch streamlit


   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ------------------------- -------------- 7.3/11.5 MB 38.8 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 36.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   -------------------------------- ------- 8.1/10.0 MB 40.8 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 38.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ---------------------- ----------------- 7.6/13.7 MB 36.6 MB/s eta 0:00:01
   ---------------------------------------- 13.7/13.7 MB 33.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/654.0 kB ? eta -:--:--
   --------------------------------------- 654.0/654.0 kB 23.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   --------------------------- ------------ 6.6/9.6 MB 30.4 MB/s eta 0:00:01
   -------------------


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

# Define the correct file path
file_path = r"C:\Users\rahul\OneDrive\Desktop\2nd Sem\Machine learning\Sematic_search\imdb_top_1000.csv"

# Load dataset
df = pd.read_csv(file_path)

# Convert columns to string and fill NaN values with an empty string
df['Series_Title'] = df['Series_Title'].astype(str).fillna("")
df['Genre'] = df['Genre'].astype(str).fillna("")
df['Overview'] = df['Overview'].astype(str).fillna("")

# Concatenate selected fields into a single text column
df['text'] = df['Series_Title'] + " " + df['Genre'] + " " + df['Overview']

# Display the first few rows
df.head()


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,text
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",The Shawshank Redemption Drama Two imprisoned ...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411","The Godfather Crime, Drama An organized crime ..."
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444","The Dark Knight Action, Crime, Drama When the ..."
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000","The Godfather: Part II Crime, Drama The early ..."
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000","12 Angry Men Crime, Drama A jury holdout attem..."


In [6]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings
df['embedding'] = df['text'].apply(lambda x: model.encode(x, convert_to_tensor=True).cpu().numpy())

df.head()  # Check embeddings


c:\Users\rahul\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\rahul\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rahul\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to ru

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,text,embedding
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",The Shawshank Redemption Drama Two imprisoned ...,"[-0.07065413, -0.011609285, -0.05595073, -0.01..."
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411","The Godfather Crime, Drama An organized crime ...","[-0.067916326, -0.0037385994, -0.08907547, 0.0..."
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444","The Dark Knight Action, Crime, Drama When the ...","[0.015399967, 0.0042574294, -0.10742431, 0.010..."
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000","The Godfather: Part II Crime, Drama The early ...","[-0.059734, 0.048530374, -0.050358385, -0.0074..."
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000","12 Angry Men Crime, Drama A jury holdout attem...","[-0.010480803, 0.03797064, -0.037760817, -0.03..."


In [24]:
import pinecone
from pinecone import Pinecone, ServerlessSpec

# ✅ Replace with your actual API key from Pinecone
api_key = "pcsk_2zkszn_UbXuT2K7parwLi3U2QEKmsw2uBr43kqgrFig4nEuC3b81CFRbeXdm98NDbc1GkR"

# Create Pinecone client
pc = Pinecone(api_key=api_key)

# Verify connection (this should return a list of your indexes)
print("Available Indexes:", [i.name for i in pc.list_indexes()])


Available Indexes: []


In [27]:
import pinecone
from pinecone import Pinecone, ServerlessSpec

# Replace with your actual Pinecone API key
api_key = "pcsk_2zkszn_UbXuT2K7parwLi3U2QEKmsw2uBr43kqgrFig4nEuC3b81CFRbeXdm98NDbc1GkR"

# Create Pinecone client
pc = Pinecone(api_key=api_key)

# Define index name
index_name = "imdb-movie-search"

# Check if index already exists
if index_name not in [i.name for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,  # Must match the embedding size of your model
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")  # ✅ Required argument
    )
    print(f"✅ Created index: {index_name}")
else:
    print(f"🔹 Index '{index_name}' already exists.")

# Connect to the index
index = pc.Index(index_name)
print("✅ Connected to Pinecone index!")


✅ Created index: imdb-movie-search
✅ Connected to Pinecone index!


In [38]:
# Upload embeddings to Pinecone
for i, row in df.iterrows():
    index.upsert([(str(i), row['embedding'], {'title': row['Series_Title'], 'Genre': row['Genre'], 'overview': row['Overview']})])

print("✅ Data uploaded to Pinecone!")


✅ Data uploaded to Pinecone!


In [39]:
def search_movies(query, top_k=5):
    # Generate query embedding
    query_embedding = model.encode(query).tolist()

    # Search in Pinecone
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    # Extract and display results
    movie_results = []
    for match in results['matches']:
        movie_results.append({
            "Title": match['metadata']['title'],
            "Overview": match['metadata']['overview'],
            "Genre": match['metadata']['Genre']
        })

    return pd.DataFrame(movie_results)


In [ ]:
# Example search query
query = "A thrilling sci-fi adventure with space battles"
search_results = search_movies(query)

# Display the search results in Jupyter Notebook or VS Code
from IPython.display import display

print("🎬 Movie Recommendations based on your query:")
display(search_results)


🎬 Movie Recommendations based on your query:


,Title,Overview,Genre
0,Airplane!,A man afraid to fly must ensure that a plane l...,Comedy
1,Zootopia,"In a city of anthropomorphic animals, a rookie...","Animation, Adventure, Comedy"
2,Kurenai no buta,"In 1930s Italy, a veteran World War I pilot is...","Animation, Adventure, Comedy"
3,Predator,A team of commandos on a mission in a Central ...,"Action, Adventure, Sci-Fi"
4,King Kong,A film crew goes to a tropical island for an e...,"Adventure, Horror, Sci-Fi"
